| Word          | Token(s)                           |
| ------------- | ---------------------------------- |
| surf          | \['surf'\]                         |
| surfing       | \['surf', '##ing'\]                 |
| surfboarding  | \['surf', '##board', '##ing'\]       |
| surfboard     | \['surf', '##board'\]               |
| snowboard     | \['snow', '##board'\]               |
| snowboarding  | \['snow', '##board', '##ing'\]       |
| snow          | \['snow'\]                         |
| snowing       | \['snow', '##ing'\]                 |

!pip install datasets

In [1]:
from datasets import load_dataset

In [3]:
import datasets
datasets.list_datasets()

['acronym_identification',
 'ade_corpus_v2',
 'adversarial_qa',
 'aeslc',
 'afrikaans_ner_corpus',
 'ag_news',
 'ai2_arc',
 'air_dialogue',
 'ajgt_twitter_ar',
 'allegro_reviews',
 'allocine',
 'alt',
 'amazon_polarity',
 'amazon_reviews_multi',
 'amazon_us_reviews',
 'ambig_qa',
 'amttl',
 'anli',
 'app_reviews',
 'aqua_rat',
 'aquamuse',
 'ar_cov19',
 'ar_res_reviews',
 'ar_sarcasm',
 'arabic_billion_words',
 'arabic_pos_dialect',
 'arabic_speech_corpus',
 'arcd',
 'arsentd_lev',
 'art',
 'arxiv_dataset',
 'ascent_kb',
 'aslg_pc12',
 'asnq',
 'asset',
 'assin',
 'assin2',
 'atomic',
 'autshumato',
 'babi_qa',
 'banking77',
 'bbaw_egyptian',
 'bbc_hindi_nli',
 'bc2gm_corpus',
 'beans',
 'best2009',
 'bianet',
 'bible_para',
 'big_patent',
 'billsum',
 'bing_coronavirus_query_set',
 'biomrc',
 'blended_skill_talk',
 'blimp',
 'blog_authorship_corpus',
 'bn_hate_speech',
 'bookcorpus',
 'bookcorpusopen',
 'boolq',
 'bprec',
 'break_data',
 'brwac',
 'bsd_ja_en',
 'bswac',
 'c3',
 'c4',


In [4]:
len(datasets.list_datasets())

1329

In [9]:
dataset = load_dataset(
    'oscar',
    'unshuffled_deduplicated_it',
    split='train[:50000]'
)

Reusing dataset oscar (/Users/jamesbriggs/.cache/huggingface/datasets/oscar/unshuffled_deduplicated_it/1.0.0/84838bd49d2295f62008383b05620571535451d84545037bb94d6f3501651df2)


In [7]:
import os

os.mkdir('./oscar')

In [12]:
from tqdm.auto import tqdm

text_data = []
file_count = 0

for sample in tqdm(dataset):
    sample = sample['text'].replace('\n', ' ')
    text_data.append(sample)
    if len(text_data) == 5_000:
        with open(f'./oscar/file_{file_count}.txt', 'w', encoding='utf-8') as fp:
            fp.write('\n'.join(text_data))
        text_data = []
        file_count += 1

100%|██████████| 50000/50000 [00:02<00:00, 18398.95it/s]


In [13]:
from pathlib import Path
paths = [str(x) for x in Path('./oscar').glob('**/*.txt')]
paths[:5]

['oscar/file_7.txt',
 'oscar/file_6.txt',
 'oscar/file_4.txt',
 'oscar/file_5.txt',
 'oscar/file_1.txt']

In [14]:
len(paths)

10

In [ ]:
!pip install tokenizers

In [15]:
from tokenizers import BertWordPieceTokenizer

In [17]:
tokenizer = BertWordPieceTokenizer(
    clean_text=True,
    handle_chinese_chars=False,
    strip_accents=False,
    lowercase=True
)

In [18]:
tokenizer.train(files=paths,
    vocab_size=10_000,
    min_frequency=2,
    special_tokens=[
        '[PAD]', '[UNK]', '[CLS]', '[SEP]', '[MASK]'],
    limit_alphabet=1000,
    wordpieces_prefix='##')

In [19]:
os.mkdir('./new_tokenizer')

tokenizer.save_model('./new_tokenizer')

['./new_tokenizer/vocab.txt']

In [20]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('./new_tokenizer')

In [21]:
tokenizer('ciao! come va?')

{'input_ids': [2, 5458, 5, 1983, 2069, 35, 3], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1]}

In [22]:
with open('./new_tokenizer/vocab.txt', 'r') as fp:
    vocab = fp.read().split('\n')

In [26]:
for i in tokenizer('ciao! come va?')['input_ids']:
    print(vocab[i], end=' ')

[CLS] ciao ! come va ? [SEP] 

In [28]:
for i in tokenizer('responsibilitá')['input_ids']:
    print(vocab[i], end=' ')

[CLS] respon ##si ##bili ##ta [SEP] 